In [3]:
from pathlib import Path
import pandas as pd
from IPython.display import display

data_dir = Path("../data")
csv_paths = sorted(data_dir.glob("*.csv"))

if len(csv_paths) != 2:
    raise FileNotFoundError(f"Expected 2 CSV files in {data_dir}, found {len(csv_paths)}.")

dataframes = {}
for csv_path in csv_paths:
    df = pd.read_csv(csv_path)
    dataframes[csv_path.stem] = df

    print(f"\n=== {csv_path.name} ===")
    feature_details = pd.DataFrame({
        "feature": df.columns,
        "dtype": df.dtypes.astype(str),
        "n_unique": [df[col].nunique(dropna=False) for col in df.columns],
        "n_missing": [df[col].isna().sum() for col in df.columns]
    })
    display(feature_details)

# Columns we want to keep (normalized names).
required = ['gender','age','physical_exercise','height','weight','dietary_preference','calories']

# Per-file mapping candidates (ordered). Add or refine candidates if your CSVs use different names.
mappings = {
    'user_nutritional_data': {
        'gender': ['gender','sex','sex_id','gender_id','gndr'],
        'age': ['age','years','user_age'],
        'physical_exercise': ['physical_exercise','physical_activity','exercise_level','activity_level'],
        'height': ['height','height_cm','stature'],
        'weight': ['weight','weight_kg','mass'],
        'dietary_preference': ['dietary_preference','diet_pref','dietary_pref','diet'],
        'calories': ['calories','calorie_goal','calorie_intake','calorie_target']
    },
    'detailed_meals_macros': {
        'gender': ['gender','sex'],
        'age': ['age'],
        'physical_exercise': ['physical_exercise','physical_activity'],
        'height': ['height','height_cm'],
        'weight': ['weight','weight_kg'],
        'dietary_preference': ['dietary_preference','diet_pref'],
        'calories': ['calories','calorie']
    }
}

def _normalize_name(name):
    return str(name).strip().lower().replace(' ', '_').replace('-', '_')

def find_col(df, candidates):
    # Build normalized map of existing columns -> actual name
    norm = { _normalize_name(c): c for c in df.columns }
    # Try candidates in order
    for cand in candidates:
        key = _normalize_name(cand)
        if key in norm:
            return norm[key]
    # fallback: partial keyword match (first match)
    for cand in candidates:
        token = _normalize_name(cand).split('_')[0]
        for n_col, orig in norm.items():
            if token and token in n_col:
                return orig
    return None

combined_parts = []
for stem, df in dataframes.items():
    selected = {}
    for req in required:
        mapped_col = None
        if stem in mappings and req in mappings[stem]:
            mapped_col = find_col(df, mappings[stem][req])
        else:
            # generic search using the required name
            mapped_col = find_col(df, [req, req.replace('_',' '), req.replace('_','-')])
        if mapped_col:
            selected[req] = df[mapped_col].copy()
        else:
            # create NA column to keep shape consistent
            selected[req] = pd.Series([pd.NA] * len(df), name=req)
    sel_df = pd.DataFrame(selected)

    # Convert likely categorical integer columns to strings for the user dataset
    if 'user_nutritional_data' in stem:
        for cat_col in ['gender', 'dietary_preference', 'physical_exercise']:
            if cat_col in sel_df.columns and pd.api.types.is_integer_dtype(sel_df[cat_col].dtype):
                sel_df[cat_col] = sel_df[cat_col].astype('string')

    # Ensure numeric columns are numeric (coerce errors to NaN)
    for num_col in ['age','height','weight','calories']:
        if num_col in sel_df.columns:
            sel_df[num_col] = pd.to_numeric(sel_df[num_col], errors='coerce')

    sel_df['source'] = stem
    combined_parts.append(sel_df)

combined = pd.concat(combined_parts, ignore_index=True)

print("\nCombined dataframe (first 5 rows):")
display(combined.head())

print("\nDtypes after alignment:")
display(combined.dtypes)

# Save combined result for downstream use
combined_path = data_dir / 'combined_users.csv'
combined.to_csv(combined_path, index=False)
print(f"\nWrote combined CSV to: {combined_path}")


=== detailed_meals_macros.csv ===


,feature,dtype,n_unique,n_missing
Ages,Ages,int64,62,0
Gender,Gender,object,2,0
Height,Height,int64,51,0
Weight,Weight,int64,71,0
Activity Level,Activity Level,object,5,0
Dietary Preference,Dietary Preference,object,4,0
Daily Calorie Target,Daily Calorie Target,int64,895,0
Protein,Protein,int64,231,0
Sugar,Sugar,float64,276,0
Sodium,Sodium,float64,231,0



=== user_nutritional_data.csv ===


,feature,dtype,n_unique,n_missing
Gender,Gender,int64,2,0
Age,Age,int64,52,0
Daily meals frequency,Daily meals frequency,int64,3,0
Physical exercise,Physical exercise,int64,5,0
Height,Height,int64,25,0
Weight,Weight,float64,99,0
BMR,BMR,float64,974,0
Carbs,Carbs,float64,1346,0
Proteins,Proteins,float64,1346,0
Fats,Fats,float64,1344,0


In [ ]:
# This cell uses the exact column names observed in the outputs above to create
# explicit per-file mappings to the required normalized columns and then
# aligns the two datasets. It saves the result to ../data/combined_users_mapped.csv
from pathlib import Path
import pandas as pd
from IPython.display import display, Markdown

data_dir = Path("../data")
csv_paths = sorted(data_dir.glob("*.csv"))

if not csv_paths:
    raise FileNotFoundError(f"No CSV files found in {data_dir}")

# load dataframes keyed by stem
dataframes = {p.stem: pd.read_csv(p) for p in csv_paths}

# Required target columns (normalized keys)
required = ['gender','age','physical_exercise','height','weight','dietary_preference','calories']

# Hand-verified mappings based on the outputs shown in the notebook
# First table (detailed_meals_macros) columns showcased included:
#  'Ages','Gender','Height','Weight','Activity Level','Dietary Preference','Daily Calorie Target', 'Calories', ...
# Second table (user_nutritional_data) included:
#  'Gender','Age','Daily meals frequency','Physical exercise','Height','Weight', 'Calories', ...

mappings = {
    'detailed_meals_macros': {
        'gender': 'Gender',
        'age': 'Ages',
        'physical_exercise': 'Activity Level',
        'height': 'Height',
        'weight': 'Weight',
        'dietary_preference': 'Dietary Preference',
        # prefer Daily Calorie Target as the user-level calorie goal if present
        'calories': 'Daily Calorie Target'
    },
    'user_nutritional_data': {
        'gender': 'Gender',
        'age': 'Age',
        'physical_exercise': 'Physical exercise',
        'height': 'Height',
        'weight': 'Weight',
        # this dataset does not list a Dietary Preference column in the observed output
        'dietary_preference': None,
        'calories': 'Calories'
    }
}

aligned_parts = []
mapping_report = []
for stem, df in dataframes.items():
    mapped = {}
    report_row = {'source': stem}
    for req in required:
        mapped_name = mappings.get(stem, {}).get(req) if stem in mappings else None
        if mapped_name and mapped_name in df.columns:
            mapped[req] = df[mapped_name].copy()
            report_row[req] = mapped_name
        else:
            # missing -> fill with NA
            mapped[req] = pd.Series([pd.NA] * len(df), name=req)
            report_row[req] = None
    sel_df = pd.DataFrame(mapped)

    # Convert categorical integer columns to strings for the user dataset
    if stem == 'user_nutritional_data':
        for cat_col in ['gender', 'physical_exercise', 'dietary_preference']:
            if cat_col in sel_df.columns and pd.api.types.is_integer_dtype(sel_df[cat_col].dtype):
                sel_df[cat_col] = sel_df[cat_col].astype('string')

    # Coerce numerics
    for num_col in ['age','height','weight','calories']:
        if num_col in sel_df.columns:
            sel_df[num_col] = pd.to_numeric(sel_df[num_col], errors='coerce')

    sel_df['source'] = stem
    aligned_parts.append(sel_df)
    mapping_report.append(report_row)

# show mapping report
report_df = pd.DataFrame(mapping_report)
print('Column mapping used for each source:')
display(report_df)

# concat and show small sample
combined_mapped = pd.concat(aligned_parts, ignore_index=True)
print('\nCombined (mapped) dataframe sample:')
display(combined_mapped.head())

print('\nDtypes after mapping:')
display(combined_mapped.dtypes)

# Save
out_path = data_dir / 'combined_users_mapped.csv'
combined_mapped.to_csv(out_path, index=False)
print(f"\nWrote mapped combined CSV to: {out_path}")